In [2]:
# imports 
from bs4 import BeautifulSoup
import requests

In [3]:
def get_productLinks(soup=''):
    return ['https://www.flipkart.com'+product_link.get('href') for product_link in soup.find_all('a',attrs={'class':'IRpwTa'})]

In [4]:
URL = 'https://www.flipkart.com/search?q=watches+for+men&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_1_6_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_1_6_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=watches+for+men&requestId=4b30e05e-0e5e-4886-82a7-181ead58bd93&as-backfill=on&page=2'
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0"}

Web_content = requests.get(URL,headers=headers).content
web_soup = BeautifulSoup(Web_content,'html.parser')
web_soup = BeautifulSoup(web_soup.prettify(),'html.parser')

# get_productLinks(web_soup)

In [5]:
def get_webpage_content(link=''):
    URL = link
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0"}

    Web_content = requests.get(URL,headers=headers).content
    web_soup = BeautifulSoup(Web_content,'html.parser')
    web_soup = BeautifulSoup(web_soup.prettify(),'html.parser')
    return web_soup


In [6]:
# main page 
URL = 'https://www.flipkart.com/search?q=watches+for+men&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_1_6_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_1_6_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=watches+for+men&requestId=4b30e05e-0e5e-4886-82a7-181ead58bd93&as-backfill=on&page=2'
# headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0"}

# Web_content = requests.get(URL,headers=headers).content
# web_soup = BeautifulSoup(Web_content,'html.parser')
# web_soup = BeautifulSoup(web_soup.prettify(),'html.parser')
main_page = get_webpage_content(URL)
product_links = get_productLinks(main_page)


In [7]:
def get_product_comment_page_main_link(product_web_page):
    try:
        if product_web_page.find('div', attrs={'class':'_3UAT2v _33R3aa'}):
            return 'https://www.flipkart.com'+product_web_page.find('div', attrs={'class':'_3UAT2v _33R3aa'}).parent.get('href')+'&page='
    except :
        return None 

In [8]:
def get_product_total_comment_pages(product_comment_web_page):
    try :
        Number_of_comment_pages = product_comment_web_page.find('div',attrs={'class':'_2MImiq _1Qnn1K'}).find('span').get_text().strip().split(' ')
        
        if len(Number_of_comment_pages) > 3:
            Number_of_comment_pages = int(Number_of_comment_pages[-1])
        else:
            Number_of_comment_pages = 1
    except:
        return 0
    
    return Number_of_comment_pages


In [9]:
def get_comments_in_page(comment_web_page):
    return [x.get_text() for x in comment_web_page.find_all('div', attrs={'class': "t-ZTKy _1QgsS5"})]

In [10]:
product_web_page = get_webpage_content(product_links[1])
comment_main_link = get_product_comment_page_main_link(product_web_page)
# get_product_total_product_comments_number(product_web_page)
product_comment_main_page = get_webpage_content(comment_main_link+str(1))
Number_of_comment_pages = get_product_total_comment_pages(product_comment_main_page)
comments = get_comments_in_page(product_comment_main_page)

In [11]:
import json 
for i in range(13,26):
    comments=[]
    print(f'Main page ------------> {i}')
    # main page 
    URL = f'https://www.flipkart.com/search?q=watches+for+men&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_1_6_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_1_6_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=watches+for+men&requestId=4b30e05e-0e5e-4886-82a7-181ead58bd93&as-backfill=on&page={i}'
    # headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0"}
    main_page = get_webpage_content(URL)
    product_links = get_productLinks(main_page)
    for product_link in product_links:
        # each product web page 
        product_web_page = get_webpage_content(product_link)
        
        comment_main_link = get_product_comment_page_main_link(product_web_page)
        # is there comment page link
        if comment_main_link:
            # get_product_total_product_comments_number(product_web_page)
            product_comment_main_page = get_webpage_content(comment_main_link+str(1))
            Number_of_comment_pages = get_product_total_comment_pages(product_comment_main_page)
            # check there are comment pages to access the comments 
            if Number_of_comment_pages:
                if Number_of_comment_pages > 60:
                    Number_of_comment_pages = 60
                # Number_of_comment_pages = 3
                for page_number in range(1,Number_of_comment_pages+1):
                    # comment web page 
                    comment_web_page = get_webpage_content(comment_main_link+str(page_number))
                    comment = get_comments_in_page(comment_web_page)
                    comments.append(comment)
                    # print(comment)
                    with open(f'data/comments_{i}.json','w') as file :
                        json.dump(comments,file )

Main page ------------> 13
Main page ------------> 14


ConnectionError: HTTPSConnectionPool(host='www.flipkart.com', port=443): Max retries exceeded with url: /sonata-np7133km03-beyond-gold-2-0-analog-watch-men/product-reviews/itm5f61c891d74f1?pid=WATFSNN8PJSDH8AG&lid=LSTWATFSNN8PJSDH8AGOAEMBF&marketplace=FLIPKART&page=6 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000017C4CAC2C50>: Failed to resolve 'www.flipkart.com' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
def main_page_productlinks(mainlink=''):
    URL = 'https://www.flipkart.com/search?q=watches+for+men&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_1_6_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_1_6_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=watches+for+men&requestId=4b30e05e-0e5e-4886-82a7-181ead58bd93&as-backfill=on&page=2'
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0"}
    
    Web_content = requests.get(URL,headers=headers).content
    web_soup = BeautifulSoup(Web_content,'html.parser')
    web_soup = BeautifulSoup(web_soup.prettify(),'html.parser')
    
    return main_page_productlinks(web_soup)

main_page_productlinks('https://www.flipkart.com/search?q=watches+for+men&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_1_6_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_1_6_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=watches+for+men&requestId=4b30e05e-0e5e-4886-82a7-181ead58bd93&as-backfill=on&page=2')

KeyboardInterrupt: 